# **Building Q_array**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def newton_raphson_system(q_value, length_value, EI = 2000):
    """
    Solve the nonlinear system using Newton-Raphson method.
    Parameters:
        q_value: int - Uniform Continuous Load Value.
        length_value: int - Length of Beam.

    Returns:
        x: array - Discretized x values.
        v: array - Solution v(x).
    """

    L  = length_value
    N  = 250
    dx = L / N
    x  = np.linspace(0, L, N + 1)
    EI = EI           # Flexural rigidity (N·m^2)
    K  = 1/EI         # Assigning value of 1/EI to K : It reduces the time to run the code
    W  = q_value      # Continuous Load (N/m) acting downward
    w  = 0.5          # Relaxation factor
    inv_dx = 1/dx
    inv_dx2 = inv_dx**2
    tol = 1e-6
    max_iter = 1500

    # init_guess == 'uniform': For Continuous Loading
    v_cont = 0.0001 * np.sign(W) * x[:]
    v_cont[0] = 0

    # Moment Equation
    M_cont = np.zeros(N+1)
    for i in range(N+1):
        M_cont[i] = W*L*x[i] - (W*(x[i]**2))/2 - (W*(L**2))/2

    # Residual Function | It is same for both Point and Continuous Load
    def residual(v, M):
        """Compute the residual vector R(v)."""
        R = np.zeros(N+1)
        v_full = np.zeros(N+1)
        v_full[:] = v
        v_full[0] = 0           # Boundary condition v(0) = 0

        # Using only FD2 Method
        for i in range(0, N+1):
            if i == 0:
                # Boundary condition dv/dx[x=0] = 0
                R[i] = ((-3*v_full[i]+4*v_full[i+1]-v_full[i+2])*inv_dx)*(1/2)
            elif i == N:
                R[i] = (2*v_full[i]-5*v_full[i-1]+4*v_full[i-2]-v_full[i-3])*inv_dx2 + (M[i]*K) * (1 + (inv_dx2)*(1/4) * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(3/2)
            else:
                R[i] = (v_full[i+1]-2*v_full[i]+v_full[i-1])*inv_dx2 + (M[i]*K) * (1 + (inv_dx2)*(1/4) * (v_full[i+1]-v_full[i-1])**2)**(3/2)
        return R

    # Jacobian Function
    def jacobian(v, M):
        """Compute the Jacobian matrix J(v)."""
        J = np.zeros((N+1, N+1))
        v_full = np.zeros(N+1)
        v_full[:] = v
        v_full[0] = 0           # Boundary condition v(0) = 0

        # Using only FD2 Method
        for i in range(0, N+1):
            if i == 0:
                # Boundary condition dv/dx[x=0] = 0
                    J[i, i], J[i, i+1], J[i, i+2] =  0, (2)*(inv_dx), (-0.5)*(inv_dx)
            elif i == N:
                J[i, i-3] = -inv_dx2
                J[i, i-2] =  4*inv_dx2 + (M[i]*K) * (3/2) * (1 + (inv_dx2)*(1/4)  * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(1/2) * (inv_dx2)*(1/4)  * 2 *(3*v_full[i]-4*v_full[i-1]+v_full[i-2]) *  (1)
                J[i, i-1] = -5*inv_dx2 + (M[i]*K) * (3/2) * (1 + (inv_dx2)*(1/4)  * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(1/2) * (inv_dx2)*(1/4)  * 2 *(3*v_full[i]-4*v_full[i-1]+v_full[i-2]) * (-4)
                J[i, i]   =  2*inv_dx2 + (M[i]*K) * (3/2) * (1 + (inv_dx2)*(1/4)  * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(1/2) * (inv_dx2)*(1/4)  * 2 *(3*v_full[i]-4*v_full[i-1]+v_full[i-2]) *  (3)
            else:
                if i == 1:
                    J[i, i-1] = 0
                else:
                  J[i, i-1] =  inv_dx2 + (M[i]*K) * (3/2) * (1 + (inv_dx2)*(1/4)  * (v_full[i+1]-v_full[i-1])**2)**(1/2) * (inv_dx2)*(1/4)  * (2*v_full[i-1]-2*v_full[i+1])
                J[i, i]   = -2*inv_dx2
                J[i, i+1] =  inv_dx2 + (M[i]*K) * (3/2) * (1 +  (inv_dx2)*(1/4)  * (v_full[i+1]-v_full[i-1])**2)**(1/2) * (inv_dx2)*(1/4)  * (2*v_full[i+1]-2*v_full[i-1])
        return J

    # Newton-Raphson iteration -- continuous load
    for iteration in range(max_iter):
        R_cont = residual(v_cont, M_cont)
        if np.linalg.norm(R_cont) < tol:
            break

        J_cont = jacobian(v_cont, M_cont)
        try:
            # Attempt to solve the system
            delta_v_cont = np.linalg.solve(J_cont, -R_cont)
        except np.linalg.LinAlgError:
            # If singular, use pseudo-inverse
            delta_v_cont = np.linalg.lstsq(J_cont, -R_cont, rcond=None)[0]    # Uses pseudo-inverse as the inverse does not exist(Using least square)

        v_cont += w*delta_v_cont

        if np.linalg.norm(delta_v_cont) < tol:
            break
    else:
        raise ValueError(f"Newton-Raphson did not converge within the maximum number of iterations = {iteration+1}.")

    # Full solution including boundaries
    v_full_cont = np.zeros(N+1)
    v_full_cont[:] = v_cont
    v_full_cont[0] = 0

    return W, L, K, x, v_full_cont

length_array = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
q_min = -1.0
q_max_list = [-10150, -1268, -376, -158.5, -81.13, -46.95, -29.58, -19.815, -13.915, -10.143]

# For each beam length, generate an array of 40 q values from q_min to the respective q_max.
q_array = [np.linspace(q_min, q_max, 40) for q_max in q_max_list]
q_array = np.array(q_array)


# **Building the 'helper' Function to calculate 'EI_min'**

In [2]:
import numpy as np

def helper(q_value, length_value, EI, tol=1e-6, N=250, max_iter=1000):

    # Discretization
    L = length_value
    dx = L / N
    x = np.linspace(0, L, N+1)
    EI =  EI
    K = 1/EI
    W = q_value
    w  =  0.5
    inv_dx = 1/dx
    inv_dx2 = inv_dx**2

    # init_guess == 'uniform': For Continuos Loading
    v_cont = 0.0001 * np.sign(W) * x[:]                         # Better than both. Cons: Jacobian becomes singular for N > 372.
    v_cont[0] = 0

    # Moment Equation
    M_cont = np.zeros(N+1)
    for i in range(N+1):
        M_cont[i] = W*L*x[i] - (W*(x[i]**2))/2 - (W*(L**2))/2

    # Residual Function | It is same for both Point and Continuous Load
    def residual(v, M):
        """Compute the residual vector R(v)."""
        R = np.zeros(N+1)
        v_full = np.zeros(N+1)
        v_full[:] = v
        v_full[0] = 0           # Boundary condition v(0) = 0

        #Using only FD2 Method
        for i in range(0, N+1):
            if i == 0:
                # Boundary condition dv/dx[x=0] = 0
                R[i] = ((-3*v_full[i]+4*v_full[i+1]-v_full[i+2])*inv_dx)*(1/2)
            elif i == N:
                R[i] = (2*v_full[i]-5*v_full[i-1]+4*v_full[i-2]-v_full[i-3])*inv_dx2 + (M[i]*K) * (1 + (inv_dx2)*(1/4) * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(3/2)
            else:
                R[i] = (v_full[i+1]-2*v_full[i]+v_full[i-1])*inv_dx2 + (M[i]*K) * (1 + (inv_dx2)*(1/4) * (v_full[i+1]-v_full[i-1])**2)**(3/2)
        return R

    # Jacobian Function
    def jacobian(v, M):
        """Compute the Jacobian matrix J(v)."""
        J = np.zeros((N+1, N+1))
        v_full = np.zeros(N+1)
        v_full[:] = v
        v_full[0] = 0           # Boundary condition v(0) = 0

        #Using only FD2 Method
        for i in range(0, N+1):
            if i == 0:
                # Boundary condition dv/dx[x=0] = 0
                    J[i, i], J[i, i+1], J[i, i+2] =  0, (2)*(inv_dx), (-0.5)*(inv_dx)
            elif i == N:
                J[i, i-3] = -inv_dx2
                J[i, i-2] =  4*inv_dx2 + (M[i]*K) * (3/2) * (1 + (inv_dx2)*(1/4)  * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(1/2) * (inv_dx2)*(1/4)  * 2 *(3*v_full[i]-4*v_full[i-1]+v_full[i-2]) *  (1)
                J[i, i-1] = -5*inv_dx2 + (M[i]*K) * (3/2) * (1 + (inv_dx2)*(1/4)  * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(1/2) * (inv_dx2)*(1/4)  * 2 *(3*v_full[i]-4*v_full[i-1]+v_full[i-2]) * (-4)
                J[i, i]   =  2*inv_dx2 + (M[i]*K) * (3/2) * (1 + (inv_dx2)*(1/4)  * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(1/2) * (inv_dx2)*(1/4)  * 2 *(3*v_full[i]-4*v_full[i-1]+v_full[i-2]) *  (3)
            else:
                if i == 1:
                    J[i, i-1] = 0
                else:
                  J[i, i-1] =  inv_dx2 + (M[i]*K) * (3/2) * (1 + (inv_dx2)*(1/4)  * (v_full[i+1]-v_full[i-1])**2)**(1/2) * (inv_dx2)*(1/4)  * (2*v_full[i-1]-2*v_full[i+1])
                J[i, i]   = -2*inv_dx2
                J[i, i+1] =  inv_dx2 + (M[i]*K) * (3/2) * (1 +  (inv_dx2)*(1/4)  * (v_full[i+1]-v_full[i-1])**2)**(1/2) * (inv_dx2)*(1/4)  * (2*v_full[i+1]-2*v_full[i-1])
        return J

    # Newton-Raphson iteration
    for _ in range(max_iter):
        R = residual(v_cont, M_cont)
        if np.linalg.norm(R) < tol:
            return True
        J = jacobian(v_cont, M_cont)
        try:
            delta = np.linalg.solve(J, -R)
        except np.linalg.LinAlgError:
            delta = np.linalg.lstsq(J, -R, rcond=None)[0]
        v_cont += w * delta
        if np.linalg.norm(delta) < tol:
            return True
    return False


# **Computing EI_min for a particular L and q**

In [ ]:
EI_arr = []
for L in range(1,11):    # Continuous Load (N/m) acting downward
    EI_sub = []
    for it in q_array[L-1]:
        W = it
        sta = 0
        end = 2000
        EI_min = 0
        while sta <= end and end - sta > 1e-4:
          mid = (sta+end)/2
          # mid = int(mid)
          print(f"Solving for length = {L}; q = {W}; EI = {mid}... \n")
          if helper(W, L, mid):
            EI_min = mid
            # print(f"EI_min = {EI_min}")
            print("Nitin_1")
            end = mid - 1e-3
          else:
            sta = mid + 1e-3
            print("Nitin_2")
        EI_min = EI_min + 1e-3
        EI_sub.append(EI_min)
        print(f"EI = {EI_min} \n")
    EI_arr.append(EI_sub)


for i in range(1,11):
    print(f"This is for length = {i}: ")
    for j in EI_arr[i-1]:
        print(j, end = ' ')
    print()


# **Building the final 'EI_array'**

In [ ]:
import math
import numpy as np

length_array = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
q_min = -1.0
q_max_list = [-10150, -1268, -376, -158.5, -81.13, -46.95, -29.58, -19.815, -13.915, -10.143]

# For each beam length, generate an array of 40 q values from q_min to the respective q_max.
q_array = [np.linspace(q_min, q_max, 40) for q_max in q_max_list]
q_array = np.array(q_array)

# For each beam length and 'q', generate an array of 10 EI values from the respective EI_min to 2000.
EI_arr = np.array([[0.06675726890563966, 16.773240650177005, 33.477816680908205, 50.18430006217957, 66.89078344345091, 83.59726682472228, 100.30375020599367, 117.01023358726502, 133.71671696853636, 150.4212929992676, 167.12777638053893, 183.83425976181027, 200.54074314308167, 217.247226524353, 233.9537099056244, 250.6582859363556, 267.36476931762695, 284.07125269889843, 300.7777360801697, 317.48421946144094, 334.1907028427124, 350.8971862239838, 367.6017622547149, 384.30824563598634, 401.01472901725776, 417.721212398529, 434.4276957798003, 451.13417916107176, 467.8406625423431, 484.54523857307424, 501.25172195434567, 517.958205335617, 534.6646887168883, 551.3711720981597, 568.077655479431, 584.7841388607025, 601.4887148914337, 618.195198272705, 634.9016816539764, 651.6081650352478], [0.39672891235351565, 13.193143686294556, 25.991465810775757, 38.78978793525696, 51.58811005973816, 64.38643218421933, 77.18284695816041, 89.9811690826416, 102.77949120712282, 115.57781333160402, 128.37613545608522, 141.17445758056638, 153.97087235450743, 166.76919447898865, 179.56751660346987, 192.36583872795106, 205.16225350189208, 217.9605756263733, 230.7588977508545, 243.55721987533573, 256.355541999817, 269.1519567737579, 281.95027889823916, 294.7486010227203, 307.5469231472016, 320.34524527168276, 333.14166004562384, 345.93998217010505, 358.7383042945862, 371.5366264190674, 384.3349485435486, 397.13136331748956, 409.92968544197083, 422.728007566452, 435.52632969093315, 448.3246518154144, 461.1229739398956, 473.92129606437686, 486.7177108383179, 499.5160329627991], [1.1329662208557127, 12.001049598693848, 22.871040327072144, 33.73912370491028, 44.60720708274841, 55.477197811126715, 66.34528118896485, 77.21527191734313, 88.08335529518126, 98.95334602355959, 109.82142940139772, 120.68951277923584, 131.55950350761412, 142.42758688545229, 153.2956702632904, 164.16566099166872, 175.035651720047, 185.90373509788515, 196.77372582626347, 207.64180920410158, 218.50989258193974, 229.379883310318, 240.24796668815614, 251.11795741653444, 261.98604079437257, 272.85603152275087, 283.72411490058903, 294.5941056289672, 305.46028165626524, 316.33027238464354, 327.20026311302183, 338.06643914031986, 348.93642986869816, 359.8064205970764, 370.67641132545475, 381.5444947032929, 392.41448543167115, 403.28256880950926, 414.15255953788756, 425.02255026626585], [2.5005365581512455, 12.590420915603639, 22.680305273056035, 32.77209698104858, 42.861981338500975, 52.95186569595337, 63.04175005340576, 73.1335417613983, 83.22151876831055, 93.31331047630312, 103.40319483375549, 113.4930791912079, 123.5829635486603, 133.67475525665284, 143.76463961410522, 153.85452397155763, 163.9463156795502, 174.0342926864624, 184.12417704391478, 194.21596875190735, 204.30585310935976, 214.3976448173523, 224.48562182426454, 234.57550618171692, 244.66539053916932, 254.75718224716192, 264.84706660461427, 274.9369509620667, 285.026835319519, 295.1186270275116, 305.208511384964, 315.29648839187627, 325.38637274932864, 335.4800718078614, 345.56804881477353, 355.6598405227661, 365.74972488021854, 375.841516588211, 385.93140094566354, 396.02128530311586], [4.762654298782349, 14.54354786872864, 24.322534088134766, 34.10342765808105, 43.88432122802735, 53.66521479797364, 63.44610836791991, 73.22509458732605, 83.0078955078125, 92.78688172721863, 102.56777529716493, 112.3486688671112, 122.1295624370575, 131.91045600700377, 141.69134957695007, 151.47033579635618, 161.2512293663025, 171.03212293624878, 180.81301650619505, 190.59391007614138, 200.3728962955475, 210.15378986549382, 219.93659078598026, 229.71748435592653, 239.49647057533264, 249.2754567947388, 259.0582577152253, 268.8372439346314, 278.6181375045776, 288.3990310745239, 298.17992464447025, 307.9570035133362, 317.7417117843628, 327.5187906532288, 337.3015915737153, 347.08248514366153, 356.865286064148, 366.6442722835542, 376.4251658535004, 386.2041520729065], [8.16536766242981, 17.78222908592224, 27.399090509414677, 37.00260047912598, 46.632813356399545, 56.24967477989197, 65.86653620338438, 75.48339762687684, 85.10216640090943, 94.71712047386171, 104.33588924789427, 113.95275067138672, 123.56961209487918, 133.18647351837157, 142.8052422924042, 152.42019636535645, 162.038965139389, 171.6558265628815, 181.2726879863739, 190.89145676040656, 200.4320241622925, 210.12327225685124, 219.7420410308838, 229.35890245437625, 238.97576387786867, 248.59071795082093, 258.20948672485355, 267.82825549888616, 277.4451169223785, 287.0600709953308, 296.67883976936344, 306.295701192856, 315.9125626163483, 325.5294240398407, 335.1481928138733, 344.76314688682555, 354.380008310318, 363.99877708435065, 373.615638507843, 383.2344072818755], [12.935651363372804, 22.399924743652345, 31.89090103149415, 41.36852586555482, 50.84615069961548, 60.32568288421631, 69.76134600639344, 79.28093255233765, 88.75855738639831, 98.23808957099916, 107.71571440505983, 117.19143188858033, 126.67096407318117, 136.14668155670168, 145.62430639076234, 155.01037839889526, 164.57955605888367, 174.05908824348452, 183.53671307754516, 193.01433791160582, 202.37179966163637, 211.97149493026737, 221.44911976432803, 230.9267445983887, 240.40246208190916, 249.73131357383733, 259.35771175003055, 268.8391512851715, 278.3129614181519, 287.7944009532929, 297.09273483657836, 306.747743270874, 316.22536810493466, 325.70490028953554, 335.1825251235962, 344.4522487487793, 354.1377747917175, 363.6173069763184, 373.09874651145935, 382.57446399497985], [19.306202167510985, 28.619794855117803, 37.90668463516235, 47.245072879791266, 56.51670385551453, 65.87225825500488, 75.13244512748719, 84.49753627967834, 93.74437169837952, 103.05033498382569, 112.43640699195862, 121.66607625579836, 130.97013219070433, 140.27609547615052, 149.6888703918457, 158.88992939758305, 168.193985332489, 177.627741104126, 186.9413337917328, 196.11759724044802, 205.42165317535404, 214.88020450401308, 224.0373944473267, 233.3414503822327, 242.649321018219, 251.95337695312503, 261.2612475891114, 270.7579459285737, 280.07153861618036, 289.38513130378715, 298.4908227825165, 307.7967860679626, 317.1008420028687, 326.40871263885504, 335.94928004074103, 345.2628727283478, 354.57646541595466, 363.6382878322602, 372.9423437671661, 382.24449235153196], [27.496365386962893, 36.62685242271423, 45.70584099388122, 54.8496794834137, 63.96109301376341, 73.07250654411314, 82.18392007446289, 91.18279992294313, 100.3361751651764, 109.37129467391969, 118.62957419586182, 127.65134225082397, 136.85240125656125, 145.86081785774232, 155.07522831726072, 164.18664184761047, 173.17598494338986, 182.4075615577698, 191.38546055030827, 200.63038861846925, 209.74180214881898, 218.70062763595584, 227.96462920951848, 237.0760427398682, 246.18745627021792, 255.1214862003327, 264.4102833309174, 273.5216968612672, 282.43665328598024, 291.7445239219665, 300.8559374523163, 309.7499130210876, 319.0787645130157, 328.1901780433655, 337.0650801067352, 346.1707715854644, 355.5244186344146, 364.6358321647645, 373.24942720413213, 382.85865922546384], [37.80368770599366, 46.66523831558228, 55.52869627571108, 64.39215423583983, 73.25370484542846, 82.11525545501708, 90.9787134151459, 99.84026402473452, 108.70181463432311, 117.56336524391176, 126.42682320404053, 135.2902811641693, 144.14992442321778, 153.01719708442687, 161.87302564239505, 170.7402983036041, 179.59994156265262, 188.46339952278137, 197.32304278182986, 206.18459339141847, 215.0442366504669, 223.9076946105957, 232.77305992126466, 241.63651788139344, 250.50569789314272, 259.35580439949035, 268.2192623596191, 277.082720319748, 285.9519003314972, 294.81726564216615, 303.6730942001343, 312.5327374591827, 321.38856601715077, 330.25774602890016, 339.1192966384888, 347.98656929969786, 356.84811990928654, 365.70776316833496, 374.5693137779236, 383.43277173805234]])
# EI_arr = [[math.ceil(val) for val in row] for row in EI_arr]
EI_arr = EI_arr + 0.1
EI_array = [[np.linspace(EI_arr[i][j], 2000, 10) for j in range(len(q_array[0]))] for i in range(len(length_array))]

delta_values_NR = []
delta_values_Euler = []

sim_num = 1
for i, L in enumerate(length_array):  # Loop over beam lengths
    q_level_NR = []
    q_level_Euler = []

    for j, q in enumerate(q_array[i]):  # Loop over q values for this length
        EI_variation_NR = []
        EI_variation_Euler = []

        for k, EI in enumerate(EI_array[i][j]):  # Loop over EI values for this (L, q)
            # Run the simulation
            print(f"Running simulation no. {sim_num}/4000 for L = {L}, q = {q}, EI = {EI}...\n")
            W, L, K, x, v_full = newton_raphson_system(q, L, EI)

            # Save displacement from Newton-Raphson
            EI_variation_NR.append(v_full[-1])

            # Save Euler-Bernoulli estimate
            delta_EB = -((W*L*x[-1]**3)*K)*(1/6) + ((W*x[-1]**4)*K)*(1/24) + ((W*L**2*x[-1]**2)*K)*(1/4)
            EI_variation_Euler.append(delta_EB)

            sim_num += 1

        q_level_NR.append(EI_variation_NR)
        q_level_Euler.append(EI_variation_Euler)

    delta_values_NR.append(q_level_NR)
    delta_values_Euler.append(q_level_Euler)

print(f"Delta Values from Newton-Raphson:\n{delta_values_NR}")
print()
print(f"Delta Values from Euler-Bernoulli:\n{delta_values_Euler}")


# **Dimensionless Plot (Least Square Method to Compute 'Alpha (α)' and 'C')**

In [4]:
# ================================
# Step 1: Gather Data for Least-Squares Fitting
# ================================

# Create empty lists to store the core nondimensional values:
X_data_NR = []  # q* core: (q * L^3 / EI)
Y_data_NR = []  # deflection* core: (δ / L)

# Loop over each beam length and corresponding q values and deflections
for i, L in enumerate(length_array):
    for j, q in enumerate(q_array[i]):
        for k, EI in enumerate(EI_array[i][j]):
            deflection = delta_values_NR[i][j][k]
            X_data_NR.append(q * (L**3) / EI)
            Y_data_NR.append(deflection / L)

# Convert lists to numpy arrays
X_data_NR = np.array(X_data_NR)
Y_data_NR = np.array(Y_data_NR)


# ================================
# Step 2: Perform Log-Log Least-Squares Fit
# ================================

# To avoid taking log of zero (or negative issues), use absolute values
valid_idx = (np.abs(X_data_NR) > 1e-15) & (np.abs(Y_data_NR) > 1e-15)
x_log = np.log(np.abs(X_data_NR[valid_idx]))
y_log = np.log(np.abs(Y_data_NR[valid_idx]))

# Perform a linear regression (polyfit) in log-log space: y_log = ln(C) + α * x_log
coeffs = np.polyfit(x_log, y_log, 1)
alpha_NR = coeffs[0]  # Slope gives α
ln_C_NR = coeffs[1]   # Intercept gives ln(C)
C_NR = np.exp(ln_C_NR)

print(f"Newton-Raphson => alpha = {alpha_NR:.4f}, C = {C_NR:.4e}")

# ================================

Newton-Raphson => alpha = 1.1220, C = 1.5713e-01


# **Log Scale Plot**

In [ ]:
# ================================
# Step 3: Create the Collapsed (Dimensionless) Plot in Log Scale
# ================================

# This plot gives — all (L, q, EI) combinations plotted together.
plt.figure(figsize=(8, 6))

q_star = []
defl_star = []

for i, L in enumerate(length_array):
    for j, q in enumerate(q_array[i]):
        for k, EI in enumerate(EI_array[i][j]):
            delta = delta_values_NR[i][j][k]
            X = q * (L**3) / EI
            Y = delta / L

            if np.abs(X) > 1e-15 and np.abs(Y) > 1e-15:
                q_star.append(C_NR * (np.abs(X) ** alpha_NR))
                defl_star.append(np.abs(Y))

plt.scatter(q_star, defl_star, color='blue', s=15, label='NR data')

plt.xscale('log')
plt.yscale('log')
plt.xlabel(r"$\left|\frac{qL^3}{EI}\right|^\alpha \times C$", fontsize=12)
plt.ylabel(r"$\left|\frac{\delta}{L}\right|$", fontsize=12)
plt.title("Collapsed Plot (All Data) - Log Scale", fontsize=14)
plt.grid(True, which="both", linestyle="--", linewidth=0.5)
plt.legend()
plt.tight_layout()
plt.show()

print()
# ================================ #
print()

# This plot gives a separate curve for each L, aggregating all points across q and EI.
plt.figure(figsize=(8, 6))

for i, L in enumerate(length_array):
    q_star = []
    defl_star = []

    for j, q in enumerate(q_array[i]):
        for k, EI in enumerate(EI_array[i][j]):
            delta = delta_values_NR[i][j][k]
            X = q * (L**3) / EI
            Y = delta / L

            if np.abs(X) > 1e-15 and np.abs(Y) > 1e-15:
                q_star.append(C_NR * (np.abs(X) ** alpha_NR))
                defl_star.append(np.abs(Y))

    plt.plot(q_star, defl_star, marker='o', linestyle='-', label=f'L = {L}')

plt.xscale('log')
plt.yscale('log')
plt.xlabel(r"$\left|\frac{qL^3}{EI}\right|^\alpha \times C$", fontsize=12)
plt.ylabel(r"$\left|\frac{\delta}{L}\right|$", fontsize=12)
plt.title("Collapsed Plot (Grouped by L) - Log Scale", fontsize=14)
plt.legend()
plt.grid(True, which="both", linestyle="--", linewidth=0.5)
plt.tight_layout()
plt.show()


# **Linear Scale Plot**

In [ ]:
# ================================
# Step 4: Create the Collapsed (Dimensionless) Plot in Linear Scale
# ================================

# This plot gives — all (L, q, EI) combinations plotted together.
plt.figure(figsize=(8, 6))

q_star = []
defl_star = []

for i, L in enumerate(length_array):
    for j, q in enumerate(q_array[i]):
        for k, EI in enumerate(EI_array[i][j]):
            delta = delta_values_NR[i][j][k]
            X = q * (L**3) / EI
            Y = delta / L

            if np.abs(X) > 1e-15 and np.abs(Y) > 1e-15:
                q_star.append(C_NR * (np.abs(X) ** alpha_NR))
                defl_star.append(np.abs(Y))

plt.scatter(q_star, defl_star, color='green', s=15, label='NR data')
plt.xlabel(r"$\left|\frac{qL^3}{EI}\right|^\alpha \times C$", fontsize=12)
plt.ylabel(r"$\left|\frac{\delta}{L}\right|$", fontsize=12)
plt.title("Collapsed Plot (All Data) - Linear Scale", fontsize=14)
plt.grid(True, linestyle="--", linewidth=0.5)
plt.legend()
plt.tight_layout()
plt.show()

print()
# ================================ #
print()

# This plot gives a separate curve for each L, aggregating all points across q and EI.
plt.figure(figsize=(8, 6))

for i, L in enumerate(length_array):
    q_star = []
    defl_star = []

    for j, q in enumerate(q_array[i]):
        for k, EI in enumerate(EI_array[i][j]):
            delta = delta_values_NR[i][j][k]
            X = q * (L**3) / EI
            Y = delta / L

            if np.abs(X) > 1e-15 and np.abs(Y) > 1e-15:
                q_star.append(C_NR * (np.abs(X) ** alpha_NR))
                defl_star.append(np.abs(Y))

    plt.plot(q_star, defl_star, marker='o', linestyle='-', label=f'L = {L}')

# Linear scale plot
plt.xlabel(r"$\left(\frac{qL^3}{EI}\right)^\alpha \times C$ (Dimensionless Load)", fontsize=12)
plt.ylabel(r"$\frac{|\delta|}{L}$ (Dimensionless Deflection)", fontsize=12)
plt.title("Collapsed Plot (Grouped by L) - Linear Scale", fontsize=14)
plt.legend()
plt.grid(True, linestyle="--", linewidth=0.5)
plt.tight_layout()
plt.show()
